In [1]:
import os
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings)
from langchain.document_loaders import ConfluenceLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from atlassian import Confluence
from InstructorEmbedding import INSTRUCTOR
import getpass
import json
import pickle
import uuid
import chromadb
import re
from chromadb.config import Settings
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

C:\Users\JackMiller\anaconda3\envs\LLM\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JackMiller\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
tokens = r"C:\Users\JackMiller\Documents\api_keys.txt"
confluence_url = 'https://c-b4web.atlassian.net/'
user = 'jackm@cb4.com'
space_key = 'ALGO'

foundation_model = "gpt-3.5-turbo"
vector_store = r"./Documents/chroma_db/"

In [68]:
with open(tokens, 'r') as file:
    content = file.read()

key_dict = json.loads(content)
atlassian_token = key_dict['atlassian']
os.environ["OPENAI_API_KEY"] = key_dict['openai']

In [51]:
confluence = Confluence(url=confluence_url,
               username=user,
               password=atlassian_token)

In [72]:
llm = ChatOpenAI(model_name=foundation_model,
                 temperature=0)

C:\Users\JackMiller\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [77]:
loader = ConfluenceLoader(
    url=confluence_url,
    username = user,
    api_key= atlassian_token)

documents = loader.load(
    space_key=space_key,
    limit=100)

In [180]:
stop = stopwords.words('english')

def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = " ".join([word for word in text.split() if word not in (stop)])
    text = re.sub("(\n+)", "", text)
    text = re.sub("(\s){2,}", " ", text)
    return text

In [177]:
for i, _ in enumerate(documents):
    documents[i].page_content = clean_text(documents[i].page_content)

In [84]:
# save locally as a checkpoint
with open('algo_confluence.pkl', 'wb') as pickle_file:
    pickle.dump(documents, pickle_file)

In [78]:
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=0)
texts = text_splitter.split_documents(documents)

token_splitter = TokenTextSplitter(chunk_size=1000,
                                  chunk_overlap=10,
                                  encoding_name="cl100k_base") #text-embedding-ada-002
tokenized_text = token_splitter.split_documents(texts)

Created a chunk of size 735, which is longer than the specified 100
Created a chunk of size 150, which is longer than the specified 100
Created a chunk of size 139, which is longer than the specified 100
Created a chunk of size 5678, which is longer than the specified 100
Created a chunk of size 238, which is longer than the specified 100
Created a chunk of size 365, which is longer than the specified 100
Created a chunk of size 204, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 261, which is longer than the specified 100
Created a chunk of size 122, which is longer than the specified 100
Created a chunk of size 185, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 1050, which is longer than the specified 100
Created a chunk of size 186, which is longer t

In [168]:
# Example of confluence page metadata
confluence.get_page_by_id('261470',
                          expand=True,
                          status=None,
                          version=None)

{'id': '261470',
 'type': 'page',
 'status': 'current',
 'title': 'Top Down Recommendations',
 'macroRenderedOutput': {},
 'extensions': {'position': 'none'},
 '_expandable': {'container': '/rest/api/space/ALGO',
  'metadata': '',
  'restrictions': '/rest/api/content/261470/restriction/byOperation',
  'history': '/rest/api/content/261470/history',
  'body': '',
  'version': '',
  'descendants': '/rest/api/content/261470/descendant',
  'space': '/rest/api/space/ALGO',
  'childTypes': '',
  'schedulePublishInfo': '',
  'operations': '',
  'schedulePublishDate': '',
  'children': '/rest/api/content/261470/child',
  'ancestors': ''},
 '_links': {'editui': '/pages/resumedraft.action?draftId=261470',
  'webui': '/spaces/ALGO/pages/261470/Top+Down+Recommendations',
  'context': '/wiki',
  'self': 'https://c-b4web.atlassian.net/wiki/rest/api/content/261470',
  'tinyui': '/x/Xv0D',
  'collection': '/rest/api/content',
  'base': 'https://c-b4web.atlassian.net/wiki'}}

## Embedding options

In [108]:
## 1. BERT (opensource)
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2") 

## 2. OpenAI - $$$
embedding_function = OpenAIEmbeddings()

##3. Instructor (instruction-Finetuned Text Embeddings) (opensource)
# model = INSTRUCTOR('hkunlp/instructor-large')
# instruction = "Represent the document for retrieval: "
# instruction_pairs = [[instruction, i.page_content] for i in tokenized_text]
# customized_embeddings = model.encode(texts_with_instructions)


## Set up vector DB to store embeddings

#### Local disk

In [110]:
# if persist_directory  and os.path.exists(vector_store):
#     vectordb = Chroma(persist_directory=vector_store, embedding_function=embedding)
# else:
vectordb = Chroma.from_documents(documents=tokenized_text, embedding=embedding)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.91s/it]


In [139]:
# get sample key 
sample_key = vectordb.get()['ids'][0]
print(sample_key)

aa47c7e3-c8d7-11ee-b31c-bcf1711ffe54


In [140]:
vectordb.get(sample_key)

{'ids': ['aa47c7e3-c8d7-11ee-b31c-bcf1711ffe54'],
 'embeddings': None,
 'metadatas': [{'id': '261337',
   'source': 'https://c-b4web.atlassian.net/spaces/ALGO/pages/261337',
   'title': 'Asymmetric * % Null'}],
 'documents': ['Status Grey Not started Stakeholders Add stakeholders Outcome What did you decide? Due date When does this decision need to be made by? Owner Background Change the CHI calculation to prefer KPIs with less nulls Action items 1 incomplete Type your task here. Use "@" to assign a user and "//" to select a due date.'],
 'uris': None,
 'data': None}

In [142]:
vectordb.get(sample_key, include=['embeddings'])

{'ids': ['aa47c7e3-c8d7-11ee-b31c-bcf1711ffe54'],
 'embeddings': [[0.010426313616335392,
   -0.01905020698904991,
   -0.017303243279457092,
   -0.04145568981766701,
   -0.02925468422472477,
   0.009733075276017189,
   -0.03449556976556778,
   0.012949703261256218,
   -0.02573302946984768,
   -0.03699123114347458,
   0.007161158137023449,
   0.01451642345637083,
   -0.010717473924160004,
   0.024817954748868942,
   0.018537210300564766,
   0.0019150725565850735,
   0.01921658404171467,
   -0.0028180163353681564,
   0.022599589079618454,
   -0.011563225649297237,
   -0.01805194281041622,
   0.00944884680211544,
   -0.01423912774771452,
   0.028561444953083992,
   -0.0156256053596735,
   0.0012053692480549216,
   0.010578826069831848,
   -0.017483485862612724,
   0.00574695086106658,
   -0.01512647420167923,
   0.004415932111442089,
   0.0006490449886769056,
   0.007258211728185415,
   -0.010883851908147335,
   -0.02301553264260292,
   0.008651621639728546,
   0.0044783237390220165,
   0.

In [147]:
len(vectordb.get(sample_key, include=['embeddings'])['embeddings'][0])

1536

#### Docker

In [ ]:
!git clone git@github.com:chroma-core/chroma.git
# docker-compose up -d --build

In [ ]:
# Or via docker

client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset()  # resets the database
collection = client.create_collection("test_collection")

for doc in tokenized_text:
    collection.add(
        ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
    )

# For langchain
vectordb = Chroma(client=client,
                  collection_name="test_collection",
                  embedding_function=embedding_function,
                 )

## Prompt engineering and querying LLM

In [115]:
retriever = vectordb.as_retriever(search_kwargs={"k":4})
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever)

In [116]:
custom_prompt_template = """You are a Confluence chatbot answering questions. 
                            Use the following pieces of context to answer the question at the end. 
                            If you don't know the answer, say that you don't know, don't try to make up an answer.

                            {context}

                            Question: {question}
                            Helpful Answer:
                            """

CUSTOMPROMPT = PromptTemplate(
    template=custom_prompt_template, input_variables=["context", "question"]
)

qa.combine_documents_chain.llm_chain.prompt = CUSTOMPROMPT

In [118]:
question = "Please explain how features are calculated in the lost sales project"

answer = qa.run(question)
print(answer)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.75it/s]


In the lost sales project, features are calculated using various formulas and data from the sales and inventory datasets. Here is an explanation of how some of the features are calculated:

1. Mean Sales: This feature is calculated by dividing the total sales by the number of days of inventory. If there are zero days of inventory, the feature is null.

2. sales_prev_1, sales_prev_2, sales_prev_3: These features represent the mean sales of the SKU-Store combination in the previous weeks. They provide information on the sales trends over time.

3. days_of_inv_prev_1, days_of_inv_prev_2, days_of_inv_prev_3: These features represent the days of inventory of the SKU-Store combination in the previous weeks. They indicate the stock levels and how they may have influenced sales.

4. Sold units products: This feature calculates the sold units of the SKU-Store combination compared to the maximum sold units of all SKU-Store combinations in the same brand and product category during the evaluation

----------------

In [188]:
config_dict = {
    "llm_name": "gpt-3.5-turbo",
    "url": 'https://c-b4web.atlassian.net/',
    "user": 'jackm@cb4.com',
    "api_key": atlassian_token,
    "space": 'ALGO'
}


class ConfluenceQA:
    def __init__(self,
                config: dict={}):
        self.config = config
        self.embedding = self.init_embeddings()
        self.llm_name = self.init_model()
        self.loader = ConfluenceLoader(url=config['url'],
                                       username = config['user'],
                                       api_key= config['api_key'])
        
        self.vectordb = None
        self.texts = None
        self.retroever = None
        self.qa = None
        
    def init_embeddings(self) -> None:
        if 'embedding' in [i.lower() for i in self.config.keys()]:
            return self.config['embedding']
        else:
            return OpenAIEmbeddings()
        
    def init_model(self) -> None:
        
        if 'llm_name' in [i.lower() for i in self.config.keys()]:
            return ChatOpenAI(model_name=self.config['llm_name'],
                              temperature=0)
        else:
            return ChatOpenAI(model_name="gpt-3.5-turbo",
                              temperature=0)
    
    def get_chunk_documents(self) -> None:      
        documents = self.loader.load(space_key=self.config['space_key'], 
                                     limit=100)

        text_splitter = CharacterTextSplitter(chunk_size=100,
                                              chunk_overlap=0)
        texts = text_splitter.split_documents(documents)
        text_splitter = TokenTextSplitter(chunk_size=1000,
                                          chunk_overlap=10,
                                          encoding_name="cl100k_base")  # This the encoding for text-embedding-ada-002
        self.texts = text_splitter.split_documents(texts)
    
    def vector_db_confluence_docs(self, force_reload:bool= False) -> None:
        self.vectordb = Chroma.from_documents(documents=self.texts,
                                              embedding=self.embedding)
        
    def retreival_qa_chain(self):
        self.retriever = self.vectordb.as_retriever(search_kwargs={"k":4})
        self.qa = RetrievalQA.from_chain_type(llm=self.llm,
                                              chain_type="stuff",
                                              retriever=self.retriever)
        
    def answer_confluence(self, question: str) -> str:
        answer = self.qa.run(question)
        return answer
    
    def run_qa_setup(self) -> None:
        self.get_chunk_documents()
        self.vector_db_confluence_docs()
        self.qa()
        
    @staticmethod
    def clean_text(text: str) -> str:
        text = text.lower()
        text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
        text = " ".join([word for word in text.split() if word not in (stop)])
        text = re.sub("(\n+)", "", text)
        text = re.sub("(\s){2,}", " ", text)
        return text

In [183]:
config_dict = {
    "llm_name": "gpt-3.5-turbo",
    "url": 'https://c-b4web.atlassian.net/',
    "user": 'jackm@cb4.com',
    "api_key": atlassian_token}

In [ ]:
confluence_qa = ConfluenceQA(config=config_dict)
confluence_qa.run_qa_setup()
result = confluence_qa.answer_confluence(question)

In [ ]:
# app.py

import streamlit as st
import os
import json
import time
from dotenv import load_dotenv

# Import the ConfluenceQA class
from confluence_qa import ConfluenceQA

try:
    from hyperplane.utils import is_jhub
    if is_jhub():
        openaiKeyFile = '/root/.secret/openai_key.json'
    else:
        openaiKeyFile = '/etc/hyperplane/secrets/openai_key.json'
    with open(openaiKeyFile) as f:
        os.environ["OPENAI_API_KEY"] = json.load(f)['openai_key']
except Exception as e:
    print(e)
    load_dotenv()

st.set_page_config(
    page_title='Q&A Bot for Confluence Page',
    page_icon='⚡',
    layout='wide',
    initial_sidebar_state='auto',
)
if "config" not in st.session_state:
    st.session_state["config"] = {}
if "confluence_qa" not in st.session_state:
    st.session_state["confluence_qa"] = None

@st.cache_resource
def load_confluence(config):
    # st.write("loading the confluence page")
    confluence_qa = ConfluenceQA(config=config)
    confluence_qa.init_embeddings()
    confluence_qa.init_models()
    confluence_qa.vector_db_confluence_docs()
    confluence_qa.retreival_qa_chain()
    return confluence_qa

with st.sidebar.form(key ='Form1'):
    st.markdown('## Add your configs')
    confluence_url = st.text_input("paste the confluence URL", "https://templates.atlassian.net/wiki/")
    username = st.text_input(label="confluence username",
                             help="leave blank if confluence page is public",
                             type="password")
    space_key = st.text_input(label="confluence space",
                             help="Space of Confluence",
                             value="RD")
    api_key = st.text_input(label="confluence api key",
                            help="leave blank if confluence page is public",
                            type="password")
    submitted1 = st.form_submit_button(label='Submit')

    if submitted1 and confluence_url and space_key:
        st.session_state["config"] = {
            "persist_directory": None,
            "confluence_url": confluence_url,
            "username": username if username != "" else None,
            "api_key": api_key if api_key != "" else None,
            "space_key": space_key,
        }
        with st.spinner(text="Ingesting Confluence..."):
            ### Hardcoding for https://templates.atlassian.net/wiki/ and space RD to avoid multiple OpenAI calls.
            config = st.session_state["config"]
            if  config["confluence_url"] == "https://templates.atlassian.net/wiki/" and config["space_key"] =="RD":
                config["persist_directory"] = "chroma_db"
            st.session_state["config"] = config

            st.session_state["confluence_qa"]  = load_confluence(st.session_state["config"])
        st.write("Confluence Space Ingested")
        

st.title("Confluence Q&A Demo")

question = st.text_input('Ask a question', "How do I make a space public?")

if st.button('Get Answer', key='button2'):
    with st.spinner(text="Asking LLM..."):
        confluence_qa = st.session_state.get("confluence_qa")
        if confluence_qa is not None:
            result = confluence_qa.answer_confluence(question)
            st.write(result)
        else:
            st.write("Please load Confluence page first.")

# zero shot topic modeling

In [ ]:
import pickle
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

zeroshot_topic_list = ["Lost sales", "Forecast", "C-retail"]

topic_model = BERTopic(
    embedding_model="thenlper/gte-small", 
    min_topic_size=15,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.85,
    representation_model=KeyBERTInspired()
)

with open(r'../algo_confluence_text.pkl', 'rb') as text:
    docs = pickle.load(text)
    
topics, probs = topic_model.fit_transform(docs)

topic_model.get_topic_info()

## DSPy

In [ ]:
import json
with open(tokens, 'r') as file:
    content = file.read()

key_dict = json.loads(content)
os.environ["OPENAI_API_KEY"] = key_dict['openai']

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

In [ ]:
# Ask any question you like to this simple RAG program.
my_question = "What castle did David Gregory inherit?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")